In [ ]:
import warnings
warnings.filterwarnings("ignore") 
from IPython.core.display import display, HTML

import time

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import make_moons
from sklearn.cluster import KMeans
from scipy.sparse import csgraph

Explorando dados

In [ ]:
random_state = 21
X_mn, y_mn = make_moons(5000, noise=.07, random_state=random_state)
print(X_mn.shape)
cmap = 'viridis'
dot_size=50

fig, ax = plt.subplots(figsize=(9,7))
ax.set_title('Data with ground truth labels ', fontsize=18, fontweight='demi')

ax.scatter(X_mn[:, 0], X_mn[:, 1],c=y_mn,s=dot_size, cmap=cmap)

## Passo 1
### Hybrid Representative Selection

In [ ]:
def getRepresentivesByRandomSelection(data, pSize):
    N = data.shape[0]
    if pSize > N:
        pSize = N
    
    selectIdx = np.random.permutation(np.arange(N))[:pSize]
    randSelect = []
    for i in selectIdx:
        randSelect.append(data[i,:])

    return np.array(randSelect)

def getRepresentativesByHybridSelection(data, pSize, cntTimes=10):
    N = data.shape[0]
    bigPSize = cntTimes * pSize
    
    if pSize > N: 
        pSize = N
    if bigPSize > N: 
        bigPSize = N

    #random selection
    np.random.seed(int(time.time()))
    RpBigPdata = getRepresentivesByRandomSelection(data, bigPSize)
    
    #KNN selection
    RpData = KMeans(n_clusters=pSize, max_iter=cntTimes).fit(RpBigPdata)
    
    return RpData

RpData = getRepresentativesByHybridSelection(X_mn, 1000, 300)

In [ ]:
RpData.cluster_centers_

## Passo 2
### Approximation of K-Nearest Representatives

In [ ]:
cntRepCls = int(np.floor(np.sqrt(RpData.cluster_centers_.shape[0])))
print(cntRepCls)
AprData = KMeans(n_clusters=cntRepCls, max_iter=600).fit(RpData.cluster_centers_)

In [ ]:
AprData.labels_.shape

In [ ]:
AprData.cluster_centers_

In [ ]:
def GaussianKernel(x, y, sigma):
    return np.exp(-(np.power(np.linalg.norm(x-y),2))/2*np.power(sigma,2))
def GaussDistance(x, y, sigma=0.5):
    return np.sqrt(GaussianKernel(x, x, sigma) - 2*GaussianKernel(x, y, sigma) + GaussianKernel(y, y, sigma))

In [ ]:
nn=AprData.predict(X_mn)

In [ ]:
N = X_mn.shape[0]
p = AprData.cluster_centers_.shape[0]

B = np.zeros((N, p))
for i in range(B.shape[0]):
    sigma  = np.mean(np.linalg.norm(X_mn[i]-AprData.cluster_centers_))
    B[i,nn[i]] = GaussianKernel(X_mn[i], AprData.cluster_centers_[nn[i]], sigma=sigma)

## Passo 3
### Bipartite Graph Partitioning

In [ ]:
B

In [ ]:
dx = np.sum(B, 1)
dx = np.where(dx == 0, 1e-10, dx)
dx = 1/dx
Dx = np.zeros((N, N))
np.fill_diagonal(Dx, dx)
Er = B.T @ Dx @ B

In [ ]:
d = np.sum(Er, 1);
d = 1/np.sqrt(d)
D = np.zeros((p, p))
np.fill_diagonal(D, d)
Dr = D @ Er @ D
Dr = (Dr + Dr.T)/2

In [ ]:
aval, avec = np.linalg.eig(Dr)

In [ ]:
idx = np.argsort(aval, kind='mergesort')[::-1]
Ncut_avec = D @ avec[:,idx[:]]

In [ ]:
res = Dx @ B @ Ncut_avec

In [ ]:
res.shape

In [ ]:
norm = (np.sqrt(np.sum(res*res,1)) + 1e-10)
for i in range(res.shape[0]):
    res[i,:] = res[i,:]/norm[i]

In [ ]:
res

In [ ]:
K =  KMeans(n_clusters=2, max_iter=1000).fit(res)

In [ ]:
pred = K.labels_

In [ ]:
fig, ax = plt.subplots(figsize=(9,7))
ax.set_title('Data after spectral clustering from scratch', fontsize=18, fontweight='demi')
ax.scatter(X_mn[:, 0], X_mn[:, 1],c=pred,s=dot_size, cmap=cmap)